# Pandas Project

*Este repositorio es un proyecto sobre la librería `pandas`de python que he realizado como tarea durante el bootcamp de Data Analytics de [Ironhack](https://www.ironhack.com/). Se trata del primer proyecto que realizamos.

Para realizarlo, hemos recibido el dataset *Global Shark Attack Incidents* disponible en [kaagle.com](https://www.kaggle.com/teajay/global-shark-attacks/version/1/kernels). Se trata de un archivo csv con datos sobre ataques de tiburones alrededor de todo el mundo.

El objetivo del proyecto es plantear una hipótesis y confirmarla con los datos del dataset. Sin embargo, los datos de la bbdd son muy caóticos, por lo que es necesario filtrar y limpiar los datos para poder confirmarla.

## Cargando el dataset

In [13]:
import pandas as pd

df = pd.read_csv("./GSAF5.csv", encoding="latin1")
display(df.head())

print(df.columns)
print(df.shape)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')
(5992, 24)


La base de datos contiene 5992 valores y 24 columnas. No obstante, muchos de las columnas contienen información poco útil (como las dos últimas, 'Unnamed: 22', 'Unnamed: 23') y la tabla contiene muchos valores nulos en algunas columnas.

In [2]:
null_cols = df.isnull().sum()
print(null_cols)

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
pdf                          0
href formula                 1
href                         3
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64


## Mi hipótesis

La hipótesis que quiero comprobar es:

> Los tiburones pequeños son más peligrosos

La base de datos no tiene una columna de tamaño. Sin embargo, la columna `Species` contiene en muchas filas información sobre el tamaño del tiburón que ha mordido.

La columna `Species` contiene muchos valores nulos y los no nulos a lo mejor no contienen datos sobre el tamaño. Por tanto, los resultados me darán una muestra que a lo mejor no es representativa de la población. Sin embargo, se trata de un ejercicio de clase y he elegido dicha hipótesis porque:

- Me parece un ejercicio interesante de regex
- Contiene datos numéricos, al que le puedo aplicar más fácilmente las funciones estadísticas que hemos visto en clase.


## Limpieza del dataset

Crearé una nueva columna `Sizes` donde guardaré los tamaños. Compararé el número de ataques y su mortalidad con los tamaños. Por lo tanto, la limpieza es sencilla: Para verificar mi hipótesis únicamente necesito las columnas tamaño y `Fatal (Y/N)`.

Para empezar, consultado el csv se observa que las columnas `Species` y `Sex` tienen un espacio después de la palabra. Normalizamos las columnas:

In [22]:
df.rename(index=str, columns={"Sex ": "Sex", "Species ": "Species"}, inplace = True)

Por otra parte, en la columna `Type` hay columnas con el valor *Invalid*. Consultando varios PDFs adjuntos en la columna `href`, se observa que dichos ataques son avisos falsos, mordeduras post-mortem o casos muy muy dudosos.

Por tanto, decido borrar todos dichos datos.

In [24]:
df=df[df["Type"] != "Invalid"].copy()

In [25]:
null_cols = df.isnull().sum()
print(null_cols)
print(df.shape)

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     38
Area                       358
Location                   445
Activity                   407
Name                       147
Sex                        494
Age                       2383
Injury                      20
Fatal (Y/N)                  5
Time                      2837
Species                   2655
Investigator or Source      12
pdf                          0
href formula                 1
href                         2
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               5472
Unnamed: 23               5472
dtype: int64
(5473, 24)


In [9]:
#df=df[df["Year"]>1900].copy()

null_cols_2 = df.isnull().sum()
print(null_cols_2)
print(df.shape)

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     22
Area                       295
Location                   363
Activity                   425
Name                       158
Sex                        504
Age                       2091
Injury                      19
Fatal (Y/N)                  9
Time                      2624
Species                   2377
Investigator or Source      13
pdf                          0
href formula                 1
href                         2
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               5310
Unnamed: 23               5310
dtype: int64
(5311, 24)
